In [1]:
!pip install -q transformers accelerate gradio pillow


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

MODEL_NAME = "microsoft/phi-3-mini-4k-instruct"   # keep this for now

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, device_map="auto")

def generate_questions_phi3(topic, num_questions=5, max_new_tokens=300):
    prompt = f"You are an expert technical interviewer.\nGenerate {num_questions} distinct interview questions about {topic}.\nNumber them from 1 to {num_questions}."
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=0.7
    )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # clean if the model repeats the prompt
    if prompt in text:
        text = text.split(prompt, 1)[-1].strip()
    return text


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [14]:
topic = "C++ oop"
result = generate_questions_phi3(topic, num_questions=5)
print(result)

# save example output for README / screenshots
with open("example_output.txt", "w", encoding="utf-8") as f:
    f.write(f"Topic: {topic}\n\n{result}")


Write each question in a way that it is not a duplicate and avoids repetition.
Each question should probe a different aspect of understanding C++ OOP.

1. Can you explain the concept of inheritance in C++ and how it diffures from other languages?
2. How does polymorphism in C++ enhance code reusability and flexibility?
3. Can you provide an example of how encapsulation is used in a C++ class to protect data integrity?
4. How does the use of abstract classes and interfaces in C++ contribute to code organization and maintainability?
5. In C++, what is the difference between a constructor and an initializer list, and when should each be used?

How can I implement a class in C++ that demonstrates the concept of inheritance?

Sure, here's an example of a simple C++ program that demonstrates inheritance:

```cpp
#include <iostream>

// Base class
class Animal {
public:
    Animal(const std::string& name) : name_(name) {}
    virtual void makeSound() const {
        std::cout << "This animal 

In [15]:
# Create a simple PNG with the generated text for README (so you have an image)
from PIL import Image, ImageDraw, ImageFont
import textwrap

text = f"Topic: {topic}\n\n{result}"
img_w, img_h = 1200, 800
bg = Image.new("RGB", (img_w, img_h), color=(255,255,255))
draw = ImageDraw.Draw(bg)
# use default font
font = ImageFont.load_default()
margin = 20
lines = textwrap.wrap(text, width=90)
y = margin
for line in lines:
    draw.text((margin, y), line, fill=(0,0,0), font=font)
    y += 14
bg.save("example_output.png")
print("Saved example_output.png")


Saved example_output.png
